# 🧪 Teste de Geração de Imagens - SYNTH_IMAGE

Este notebook testa a geração de imagens sintéticas de pastagens brasileiras no Google Colab.

---

## 📦 Instalação e Configuração

In [ ]:
# Instalar dependências faltantes
!pip install controlnet-aux ultralytics xformers --upgrade --quiet

# Verificar instalação GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM disponível: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 🔧 Importações

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Configurar device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando device: {device}")

## 🤖 Carregamento do Modelo

In [ ]:
# Carregar modelo Stable Diffusion
model_id = "runwayml/stable-diffusion-v1-5"

print("Carregando modelo Stable Diffusion...")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False
).to(device)

# Usar scheduler mais rápido
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Otimização para Colab
if device == "cuda":
    pipe.enable_memory_efficient_attention()
    pipe.enable_model_cpu_offload()

print("Modelo carregado com sucesso!")

## 🎨 Geração de Imagens de Teste

In [ ]:
# Prompts para pastagens brasileiras
prompts = [
    "aerial view of brazilian pasture with cattle, green grass, realistic, high quality",
    "tropical grassland in brazil, savanna landscape, cattle grazing, blue sky, photorealistic",
    "brazilian ranch pasture, green fields, farm animals, countryside, detailed",
    "cerrado landscape brazil, grass field, scattered trees, natural lighting"
]

# Parâmetros de geração
generator = torch.Generator(device=device).manual_seed(42)

# Gerar imagens
images = []
for i, prompt in enumerate(prompts[:2]):  # Gerar apenas 2 para teste
    print(f"Gerando imagem {i+1}/2: {prompt[:50]}...")
    
    image = pipe(
        prompt=prompt,
        negative_prompt="low quality, blurry, distorted, ugly, bad anatomy",
        num_inference_steps=20,  # Reduzido para velocidade
        guidance_scale=7.5,
        width=512,
        height=512,
        generator=generator
    ).images[0]
    
    images.append(image)
    
print("Imagens geradas com sucesso!")

## 📸 Visualização dos Resultados

In [ ]:
# Exibir imagens geradas
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

for i, (image, prompt) in enumerate(zip(images, prompts[:2])):
    axes[i].imshow(image)
    axes[i].set_title(f"Imagem {i+1}\n{prompt[:40]}...", fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Salvar imagens
for i, image in enumerate(images):
    image.save(f"pastagem_brasileira_{i+1}.png")
    print(f"Imagem {i+1} salva como: pastagem_brasileira_{i+1}.png")

## 🎯 Teste ControlNet (Opcional)

In [ ]:
# Testar ControlNet se disponível
try:
    from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
    from controlnet_aux import CannyDetector
    import cv2
    
    print("✅ ControlNet disponível!")
    
    # Criar uma imagem de borda simples para teste
    canny = CannyDetector()
    
    # Usar primeira imagem gerada como base
    if images:
        canny_image = canny(images[0])
        
        # Exibir resultado
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(images[0])
        plt.title("Imagem Original")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(canny_image, cmap='gray')
        plt.title("Bordas Canny")
        plt.axis('off')
        plt.show()
        
        print("ControlNet testado com sucesso!")
    
except ImportError as e:
    print(f"❌ ControlNet não disponível: {e}")
except Exception as e:
    print(f"⚠️ Erro no teste ControlNet: {e}")

## 🎯 Teste YOLO (Opcional)

In [ ]:
# Testar YOLO se disponível
try:
    from ultralytics import YOLO
    
    print("✅ YOLO disponível!")
    
    # Carregar modelo pré-treinado
    model = YOLO('yolov8n.pt')
    
    # Testar detecção na primeira imagem
    if images:
        results = model(images[0])
        
        # Exibir resultados
        annotated_frame = results[0].plot()
        
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
        plt.title("Detecções YOLO")
        plt.axis('off')
        plt.show()
        
        print("YOLO testado com sucesso!")
    
except ImportError as e:
    print(f"❌ YOLO não disponível: {e}")
except Exception as e:
    print(f"⚠️ Erro no teste YOLO: {e}")

## 📊 Métricas de Performance

In [ ]:
# Informações de memória
if torch.cuda.is_available():
    print("📊 Uso de Memória GPU:")
    print(f"Alocada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Reservada: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    print(f"Máxima alocada: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

# Verificar tamanhos das imagens
if images:
    print("\n📸 Informações das Imagens:")
    for i, image in enumerate(images):
        print(f"Imagem {i+1}: {image.size} - Modo: {image.mode}")

print("\n✅ Teste concluído com sucesso!")
print("\n🎯 Próximos passos:")
print("- Ajustar prompts para pastagens brasileiras específicas")
print("- Implementar pipeline completo com ControlNet")
print("- Integrar detecção YOLO para validação")
print("- Configurar dataset personalizado")